# Load Packages

In [1]:
import numpy as np

from estimation.robust_estimator import SampleMean, TruncatedMean, MedianofMean, CatoniMean, WeaklyRobustMean
from estimation.heavy_tail_observations import weibull_observation, frechet_observation, pareto_observation, gaussian_observation

# Run Experiments

In [2]:
seed = 1
samples = 5000
steps = 100
noise_type = 'pareto'
p = 1.9
scale = 1.0
mean = 1.0

if noise_type == 'weibull':
    alpha = p
    nu = scale**p
    
    get_observation = lambda : weibull_observation(mean,scale,alpha)
elif noise_type == 'frechet':
    alpha = p+0.05
    nu = scale**p*gamma(1-p/alpha)
    
    get_observation = lambda : frechet_observation(mean,scale,alpha)
elif noise_type == 'pareto':
    alpha = p+0.05
    nu = alpha*scale**alpha/(alpha-p)
    
    get_observation = lambda : pareto_observation(mean,scale,alpha)
elif noise_type == 'gaussian':
    nu = scale**2
    
    get_observation = lambda : gaussian_observation(mean,scale)
    
sample_mean = SampleMean(nu, p)
# trunc_mean = TruncatedMean(nu, p)
trunc_mean = TruncatedMean(nu, p, delta=1.0, schedule=True)
# median_mean = MedianofMean(nu,p)
median_mean = MedianofMean(nu,p, delta=1.0, schedule=True)
# catoni_mean = CatoniMean(nu,p)
catoni_mean = CatoniMean(nu,p, delta=1.0, schedule=True)
weakly_robust_mean = WeaklyRobustMean(nu,p)

sample_mean_error_list = []
trunc_mean_error_list = []
median_mean_error_list = []
catoni_mean_error_list = []
weakly_robust_mean_error_list = []

np.random.seed(seed)
for i in range(samples):
    y = get_observation()

    sample_mean.update(y)
    trunc_mean.update(y)
    median_mean.update(y)
    catoni_mean.update(y)
    weakly_robust_mean.update(y)

    if ((i+1)%steps)==0 or i==0:
        y_hat = sample_mean.predict()
        sample_mean_error_list.append(np.abs(mean-y_hat))
        y_hat = trunc_mean.predict()
        trunc_mean_error_list.append(np.abs(mean-y_hat))
        y_hat = median_mean.predict()
        median_mean_error_list.append(np.abs(mean-y_hat))
        y_hat = catoni_mean.predict()
        catoni_mean_error_list.append(np.abs(mean-y_hat))
        y_hat = weakly_robust_mean.predict()
        weakly_robust_mean_error_list.append(np.abs(mean-y_hat))
        
print("Noise - {}, Moment - {:.2f}, Nu = {:.2f}".format(noise_type, p, nu))
print("Sample Mean Error : {:.3f}".format(sample_mean_error_list[-1]))
print("Truncated Mean Error : {:.3f}".format(trunc_mean_error_list[-1]))
print("Median of Mean Error : {:.3f}".format(median_mean_error_list[-1]))
print("Catoni's Mean Error : {:.3f}".format(catoni_mean_error_list[-1]))
print("Weakly Robust Mean Error : {:.3f}".format(weakly_robust_mean_error_list[-1]))

/home/dlxhrl/bandit/estimation/robust_estimator.py:114: RuntimeWarning: invalid value encountered in double_scalars
  remainder = lambda x: np.sum(_cal_psi(alpha*(np.array(self._y_list) - x),self._p))/alpha/n


Noise - pareto, Moment - 1.90, Nu = 39.00
Sample Mean Error : 0.055
Truncated Mean Error : 0.055
Median of Mean Error : 0.092
Catoni's Mean Error : 0.055
Weakly Robust Mean Error : 0.055
